In [1]:
from ehrmonize import EHRmonize

In [2]:
models = [
    'gpt-3.5-turbo',
    # 'gpt-4',
    'gpt-4o',
    # 'meta.llama2-13b-chat-v1',
    # 'meta.llama2-70b-chat-v1',
    'meta.llama3-70b-instruct-v1:0',
    'anthropic.claude-v2:1',
    'anthropic.claude-instant-v1',
    # 'mistral.mistral-7b-instruct-v0:2',
    'mistral.mixtral-8x7b-instruct-v0:1',
]

In [5]:
for m in models:
      ehrm = EHRmonize(m, temperature=1)

      results, decision, consistency = ehrm.clean_route(
            route='through the mouth',
            classes=['intravenous', 'rectal','other'],
            n_attempts=3,
            agentic=False
          )

      print(f"Model: {m}")
      print(f"Results: {results}")
      print(f"Decision: {decision}")
      print(f"Consistency: {consistency:.2f}")
    

Model: gpt-3.5-turbo
Results: ['rectal', 'other', 'other']
Decision: other
Consistency: 0.67
Model: gpt-4o
Results: ['other', 'other', 'other']
Decision: other
Consistency: 1.00
Model: meta.llama3-70b-instruct-v1:0
Results: ['other', 'other', 'other']
Decision: other
Consistency: 1.00
Model: anthropic.claude-v2:1
Results: ['rectal', 'rectal', 'rectal']
Decision: rectal
Consistency: 1.00
Model: anthropic.claude-instant-v1
Results: ['rectal', 'other', 'other']
Decision: other
Consistency: 0.67
Model: mistral.mixtral-8x7b-instruct-v0:1
Results: ['other', 'other', 'other']
Decision: other
Consistency: 1.00


In [6]:
for m in models:
      ehrm = EHRmonize(m, temperature=1)

      results, decision, consistency = ehrm.get_generic_name(
            drugname='ativan',
            n_attempts=3,
            agentic=True
          )

      print(f"Model: {m}")
      print(f"Results: {results}")
      print(f"Decision: {decision}")
      print(f"Consistency: {consistency:.2f}")
    

Model: gpt-3.5-turbo
Results: ['lorazepam', 'lorazepam', 'generic name: lorazepam']
Decision: lorazepam
Consistency: 0.67
Model: gpt-4o
Results: ['lorazepam', 'lorazepam', 'lorazepam']
Decision: lorazepam
Consistency: 1.00
Model: meta.llama3-70b-instruct-v1:0
Results: ['lorazepam', 'lorazepam', 'lorazepam']
Decision: lorazepam
Consistency: 1.00
Model: anthropic.claude-v2:1
Results: ['lorazepam', 'lorazepam', 'lorazepam']
Decision: lorazepam
Consistency: 1.00
Model: anthropic.claude-instant-v1
Results: ['lorazepam', 'lorazepam', 'lorazepam']
Decision: lorazepam
Consistency: 1.00
Model: mistral.mixtral-8x7b-instruct-v0:1
Results: ['lorazepam', 'lorazepam', 'lorazepam']
Decision: lorazepam
Consistency: 1.00


In [9]:
for m in models:
      ehrm = EHRmonize(m, temperature=1)

      results, decision, consistency = ehrm.classify_drug(
            drugname='ativan/vancocin',
            classes=['antibiotic', 'anticoagulant', 'other'],
            n_attempts=3,
            agentic=False
          )

      print(f"Model: {m}")
      print(f"Results: {results}")
      print(f"Decision: {decision}")
      print(f"Consistency: {consistency:.2f}")
    

Model: gpt-3.5-turbo
Results: ['other', 'other', 'other']
Decision: other
Consistency: 1.00
Model: gpt-4o
Results: ['other', 'other', 'other']
Decision: other
Consistency: 1.00
Model: meta.llama3-70b-instruct-v1:0
Results: ['other', 'other', '[antibiotic, other]']
Decision: other
Consistency: 0.67
Model: anthropic.claude-v2:1
Results: ['antibiotic', 'antibiotic', 'antibiotic']
Decision: antibiotic
Consistency: 1.00
Model: anthropic.claude-instant-v1
Results: ['other', 'other', 'other']
Decision: other
Consistency: 1.00
Model: mistral.mixtral-8x7b-instruct-v0:1
Results: ['other', 'other  conf', 'other  confidence:']
Decision: unsure
Consistency: 0.33
